<a href="https://colab.research.google.com/github/VemareddyDaram/NLP_Assignment2_EDC_Vemareddy_Daram/blob/main/SentimentAnalysis_TransformersMethod_Vemareddy_Daram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch pandas nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from transformers import pipeline
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv('/content/sample_data/tweets-data.csv').sample(n=500, random_state=42)

**Step 1: Clean tweets**

In [ ]:
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
    tokens = text.lower().split()
    cleaned = [word for word in tokens if word not in stop_words]
    return " ".join(cleaned)

df['cleaned_tweet'] = df['Tweets'].astype(str).apply(clean_tweet)

**STEP 2: Load transformers sentiment pipeline **

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


STEP 3 Apply pipeline

In [ ]:
results = sentiment_pipeline(df['cleaned_tweet'].tolist(), truncation=True)

**Step 4: Add results to dataframe**

In [ ]:
df['sentiment_label'] = [res['label'].lower() for res in results]
df['sentiment_score'] = [res['score'] for res in results]

**Step 5: View result**

In [ ]:
print(df[['Tweets', 'cleaned_tweet', 'sentiment_label', 'sentiment_score']].head())

                                                 Tweets  \
2899  Le #DessinDePresse de Sanaga : ls sont morts c...   
594   #Russia #Wagner #RussiaCivilWar https://t.co/P...   
2870  Exclusive content -https://t.co/oEiSIIB2Z1\n.\...   
52    Auch heute geht die politische Nachricht des T...   
1391  @crazyclipsonly Same type that would take a ho...   

                                          cleaned_tweet sentiment_label  \
2899  le dessindepresse de sanaga ls sont morts comm...        negative   
594                        russia wagner russiacivilwar        negative   
2870  exclusive content cosplay japan titan titanics...        negative   
52    auch heute geht die politische nachricht des t...        negative   
1391  type would take homemade playstationcontrolled...        negative   

      sentiment_score  
2899         0.981537  
594          0.962062  
2870         0.961531  
52           0.975570  
1391         0.993684  


In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

Device set to use cpu


In [ ]:
label_map = {
    "NEGATIVE": "negative",
    "NEUTRAL": "neutral",
    "POSITIVE": "positive"
}
df['sentiment_label'] = [label_map[res['label']] for res in results]
df['sentiment_score'] = [res['score'] for res in results]

In [ ]:
print(results[:3])

[{'label': 'NEGATIVE', 'score': 0.9815365672111511}, {'label': 'NEGATIVE', 'score': 0.9620620608329773}, {'label': 'NEGATIVE', 'score': 0.9615312218666077}]
